The Tennis Tournament in 2019
-- 

Introduction

- Our group's topic is about the tennis tournaments and players involved. There are many players from different regions with a wide range of ages and skills competing to win. These players are ranked based on their capabilities.

- In the data analysis, our group wants to predict **How a player's age, aces and serve points relates to the number of matches they won in 2019's tournaments**.
  
- The dataset is about **Game Results for Top 500 Players from 2017-2019** for many tennis tournaments, which contains various tennis player's information such as rank, age, etc. And for our prediction, we are only containing the **All tournaments** in **2019** for answering the question. We are using the tournament date, winner's average age, name, aces, and serve points. 

In [ ]:
#Premilinary Data Analysis
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)

In [ ]:
atp <- read_csv("data/atp2017-2019.csv") |>
    select(tourney_date, winner_name, winner_age, w_ace, w_svpt) |>
    filter(tourney_date > 2019000)

In [ ]:
#We each row of the data frame to have the winner's name, and a column for thier aces, serve points, wins
# and average ages.

#Create a data frame with the total aces and serve points for each player
atp_points <- atp |>
    group_by(winner_name) |>
    summarize(aces = sum(w_ace), serve_points = sum(w_svpt))

#Create a vector with the number of wins for each player
atp_wins <- atp |>
    group_by(winner_name) |>
    summarize(wins = n()) |>
    pull(wins)

#Create a vector with the average age of each player. Since the age variable in this dataset has many
#decimal places take the average age of a player over the year.
atp_age <- atp |>
    group_by(winner_name) |>
    summarize(age = mean(winner_age)) |>
    mutate(age = round(age, 0)) |>
    pull(age)

#Combine into one data frame
atp_data <-  bind_cols(atp_points, wins = atp_wins) |>
    bind_cols(age = atp_age)
#atp_data

In [ ]:
#Split the data
set.seed(1)
atp_split <- initial_split(atp_data, prop = 0.75, strata = wins)
atp_training <- training(atp_split)
atp_testing <- testing(atp_split)

In [ ]:
#Constructing useful table out of training data (number of observations / missing values for each variable)
atp_missing <- count(atp_training, name = "observations") |>
    mutate(na_vals = sum(is.na(atp_training)))
atp_missing

#Age distribution visualization
options(repr.plot.height = 7, repr.plot.width = 11)
age_hist <- ggplot(atp_training, aes(x = age)) +
    geom_histogram(binwidth = 2) +
    labs(x = "Age (years)", y = "Count",
         title = "Distribution of Ages in \n 2019 Tennis Tournaments") +
    theme(text = element_text(size = 15))
win_hist <- ggplot(atp_training, aes(x = wins)) +
    geom_histogram(binwidth = 20) +
    labs(x = "Wins", y = "Count",
     title = "Distribution of Matches Won by a \n Single Player in 2019 Tennis Tournaments") +
    theme(text = element_text(size = 15))
plot_grid(age_hist, win_hist, ncol = 2)

In [ ]:
# Define the model specification for linear regression
linear_reg_spec <- linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

# Define the formula for the model
formula <- wins ~ age + aces + serve_points

# Fit the linear regression model
linear_reg_fit <- linear_reg_spec |>
  fit(formula, data = atp_training)

# Predict on the training data
predictions <- predict(linear_reg_fit, new_data = atp_training) |>
  bind_cols(atp_training) |>
  rename(predicted_wins = .pred)

#------Note! This is for method and result people------------

#Extract out cofficients out (ages, serve points, aces)
model_coefficients <- summary(linear_reg_fit$fit)$coefficients
model_coefficients
# Here's a note for method people:
# For age esimate you saw in the small table below:
# A negative coefficient would indicate that an increase in age is associated with fewer wins.
# Similar works for aces estimate and sever points estimate:
# A positive value means that more aces results in an increase in the number of wins.
# for serve points, positive means winning more serve points is good for the number of matches won,
# they can work in another way around means if it's negative then means more ... (aces, serve points) is actually not good in ...(wins)
#you can delete this part after 
#--------------------------------------------------------------

# Actual vs Predicted Wins Visualization
wins_comparsion_plot <- ggplot(predictions, aes(x = wins, y = predicted_wins)) +
                        geom_point(alpha = 0.5) +
                        geom_abline(slope = 1, intercept = 0, col = "red", linetype = "dashed") +
                        labs(x = "Actual Wins (Num)", y = "Predicted Wins (Num)",
                        title = "Predicted vs Actual Wins based on Aces and Serve Points") +
                        theme_minimal()

#------Another Note for the above visualization--------
# This visualization *wins_comparsion_plot* is for the predictive wins made by the linear regression model
# this is the relation between the actual wins of players and the prediction of the wins
# Why is this form? Because i have tried a lot of variants and the aces and serve points did
# not actually contribute that much in the win rates, so i've decided to merge them into one
# graph of the prediction of the wins. 

# Each POINT on the graph represents an individual player. 
# The x is the number of matches the player actually won in 2019,
# and the y is the number of matches the regression model predicts
# they should have won based on their age, aces, and serve points.

# For the RED DASHED LINE: This is the line of perfect prediction. 
# If every point fell on this line, it would mean that the model's predictions were exactly accurate (which is not)
# I used dashed line because it would make the graph easier to look

# For Points' Distribution: how the points spread around the red dashed line indicates the model's accuracy. 
# Points close to the line suggest accurate predictions, while points far from the line means just less accurate.
#you can delete this note after
#---------Note Ends here---------------------------------

# Analysis to visualize how age affects wins
# Calculate average aces and serve points for use in predictions
avg_aces <- mean(atp_training$aces, na.rm = TRUE)
avg_serve_points <- mean(atp_training$serve_points, na.rm = TRUE)

# Generate a new data frame for prediction across a range of ages
age_range <- data.frame(age = seq(min(atp_training$age, na.rm = TRUE), 
                                  max(atp_training$age, na.rm = TRUE), 
                                  length.out = 100),
                        aces = avg_aces,
                        serve_points = avg_serve_points)

# Predict wins using the model across the age range
age_predictions <- predict(linear_reg_fit, new_data = age_range) |>
  bind_cols(age_range) |>
  rename(predicted_wins = .pred)

# Plot age against predicted wins
ages_plot <- ggplot(age_predictions, aes(x = age, y = predicted_wins)) +
                              geom_line() +
                              scale_x_continuous(breaks = seq(from = floor(min(age_predictions$age)), 
                                  to = ceiling(max(age_predictions$age)), 
                                  by = 1)) +
                              labs(x = "Age (years)", y = "Predicted Wins (Num)",
                              title = "Predicted Wins vs. Player Age") +
                              theme_minimal()

# Add an overall title
combined_plot <- plot_grid(wins_comparsion_plot, ages_plot, ncol = 2)
overall_title <- ggdraw() + 
  draw_label("A player's Aces, Serve Points (Left) and Ages (Right) relates to the number of matches they won in 2019", 
             fontface = 'bold', x = 0.5, hjust = 0.5) +
  theme(plot.margin = margin(0, 0, 0, 7))
final_plot <- plot_grid(overall_title, combined_plot, ncol = 1, rel_heights = c(0.1, 1))
final_plot

**Methods**

To investigate the expected wins across several age groups for major tennis tournaments in 2019, we will employ a regression model. To do this we will split the data into a training and testing set to ensure the model is accurat. We will use K-nearest neighbours regression to construct a model, train it with the training data set, and evaluate its performance using cross-validation.

For this analysis, we will focus on the following columns from the data set based in 2019: tourney_date, winner_a, w_ace, w_svpt. Using the predictor variables age, aces, and serve points the regression model will predict the expected number of wins a player will get based on their age, aces and number of serve points.



**Expected outcomes and significance**

We anticipate our regression model will accurately predict the expected number of wins players will get across a variety of age groups in major tournaments. 

Understanding ese valuesewillan provide valuable insigts into the performance of athletewith n regard to their age. This model can aid players, coaches, and analysts in strategizing training regimes, player selection, and scheduling.

Future investigations could explore the impact of additional variables to create more accuracy in determining expected winsrate of an age group suchthe number of double faults or saved break pointsaces with respect to age. Furthermore, extending the set to include other major tournaments in the from several years could a more comprehensive understanding of age-related performance trends in professional tennis.
